In [149]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from fitter import Fitter, get_common_distributions, get_distributions
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statistics
import math
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import random

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)

#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
    newUserTitles.append(temp1)
    newUserRatings.append(temp2)
    


#this might lead to a referencce error
#populate infos; a list of data frames
infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        tmp.append(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
    combinedFeatures.append(tmp)


def transform1(arr):
    ret = []
    for item in arr:
        if(item<=.5):
            ret.append(1/item)
        if(item>.5):
            ret.append(-1*(1/(1-item)))
    return ret

def transform2(arr):
    ret = []
    for item in arr:
        ret.append(1/item)
    return ret

X = []
y = []

for i in range(len(combinedFeatures)):
    count_matrix = CountVectorizer().fit_transform(combinedFeatures[i]).toarray().tolist()
    randIndex = random.randint(0, len(count_matrix)-1)
    randTestItem = count_matrix[randIndex]
    del count_matrix[randIndex]


    cosine_sim = cosine_similarity(X = count_matrix ,Y = [randTestItem])

    ratings = newUserRatings[i]
    randomRating = ratings[randIndex]
    del ratings[randIndex]

    print("p1",len(count_matrix))
    print("p2",len(cosine_sim))
    print(cosine_sim)
    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))
    print(similairities)

    averageRating =  sum(ratings)/(len(ratings))

    tmp = []
    tmp.append(averageRating)
    tmp.append(sum(similairities)/(len(similairities)))
    tmp.append(statistics.variance(ratings))
    tmp.append(statistics.variance(similairities))
    ratingsZ = stats.zscore(ratings)
    ratingsZ = [0 if math.isnan(x) else x for x in ratingsZ]
    ratingsZ = [-item if item<0 else item for item in ratingsZ]
    simsZ  = stats.zscore(similairities)
    simsZ = [-item if item<0 else item for item in simsZ]

    totalRating = 0

    #does adding constant multiplier help?
    #nots: 20
    for sim, rating,simZ, ratingZ in zip(similairities, ratings, simsZ, ratingsZ):
        totalRating += sim*(rating-averageRating)*simZ*ratingZ
    
    tmp.append(totalRating)
    #next try just total rating
    X.append(tmp)
    y.append(randomRating)



#new varriables to feed into the linear reg model:
#variance of ratings, variance of similairities
#mean of ratings, mean of similarities, rating diff

#new model:

# regr = MLPRegressor(hidden_layer_sizes=(200, 100), max_iter = 10000).fit(X, y)
# print("after nearal net regression:", regr.score(X, y))
# predicted = regr.predict(X)
# for item, actual in zip(predicted, y):
#     print("predicted:", item)
#     print("actual:", actual)


#old model
# reg = LinearRegression().fit(X, y)
# #note: uses training data to test
# print("after linear regression:", reg.score(X, y))
# predicted = reg.predict(X)
# for item, actual in zip(predicted, y):
#     print("predicted:", item)
#     print("actual:", actual)









p1 143
p2 143
[[0.04662524]
 [0.        ]
 [0.04089304]
 [0.        ]
 [0.03940552]
 [0.09100315]
 [0.04445542]
 [0.        ]
 [0.        ]
 [0.04662524]
 [0.13650473]
 [0.        ]
 [0.04347826]
 [0.        ]
 [0.        ]
 [0.17025131]
 [0.13336627]
 [0.        ]
 [0.12510865]
 [0.08512565]
 [0.12510865]
 [0.03686049]
 [0.04170288]
 [0.        ]
 [0.        ]
 [0.08891084]
 [0.        ]
 [0.2173913 ]
 [0.08512565]
 [0.03745029]
 [0.04347826]
 [0.        ]
 [0.04445542]
 [0.03940552]
 [0.        ]
 [0.09100315]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.08512565]
 [0.13043478]
 [0.08178608]
 [0.1522774 ]
 [0.0725954 ]
 [0.04662524]
 [0.08025724]
 [0.04662524]
 [0.12267912]
 [0.03806935]
 [0.17025131]
 [0.        ]
 [0.04089304]
 [0.04256283]
 [0.17391304]
 [0.        ]
 [0.        ]
 [0.04256283]
 [0.07744031]
 [0.07881104]
 [0.        ]
 [0.13043478]
 [0.11616046]
 [0.12768848]
 [0.        ]
 [0.        ]
 [0.03806935]
 [0.        ]
 [0.08178608]
 [0.04445542]
 [0.04445542]
 [0.04

In [156]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)


#another feature can be average user score for the movie in question
#how can this integrate the other methods to recommend simlair movies
#rather than directly predict what the user rates
#should there be words omited from the descriptions to reduce sudo simlairtiy
#look into inverse document frequency

#https://www.youtube.com/watch?v=O9aAwvk6SNI&ab_channel=HackersRealm

regr = MLPRegressor(hidden_layer_sizes=(1000,  1000, 1000), 
                    activation = "relu",
                    solver ="adam",
                    learning_rate_init =.00001,
                    max_iter = 10000,
                    n_iter_no_change = 100).fit(X_train, y_train)

score  = regr.score(X_test, y_test)
file = open("testing.txt", "a")
file.write("hidden_layer_sizes="+str(regr.hidden_layer_sizes)+", max_iter = "+str(regr.n_iter_)+" " + str(score)+"\n")
file.close()

print("after nearal net regression:", score)
print(regr.n_iter_)
print(regr.hidden_layer_sizes)

predicted = regr.predict(X_test)
for item, actual in zip(predicted, y_test):
    print("predicted:", item)
    print("actual:", actual)

after nearal net regression: -0.31749215720931034
10000
(1000, 1000, 1000)
predicted: 3.9035727913697587
actual: 3.0
predicted: 4.670201243266667
actual: 4.0
predicted: 4.822735110400296
actual: 4.0
predicted: 3.3847751026912705
actual: 5.0
predicted: 3.682444423657722
actual: 3.5
predicted: 3.6006656016675462
actual: 3.0
predicted: 4.056034277896135
actual: 4.0
predicted: 4.574822945055988
actual: 5.0
predicted: 2.9970633225035086
actual: 5.0
predicted: 3.4732412995539166
actual: 3.0
predicted: 3.689169108588059
actual: 4.0
predicted: 3.9450241199160385
actual: 4.0
predicted: 3.278853593880186
actual: 4.0
predicted: 3.333002808892066
actual: 3.0
predicted: 3.3137764859631726
actual: 3.0
predicted: 2.9622937157158935
actual: 3.0
predicted: 3.4471108025506583
actual: 3.0
predicted: 3.9031463156278314
actual: 4.0
predicted: 4.127857851857565
actual: 5.0
predicted: 2.570898114415924
actual: 2.0
predicted: 1.9185487531534173
actual: 1.0
predicted: 3.893212746663771
actual: 4.0
predicted: 2

d:\repos\movie_rec_proj\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
